# Comparison

The distance between two fingerprints is calculated in two steps:

1. Calculate __feature distances__: Calculate distance for each feature (e.g. distance between pairwise 85 size feature bits). Use L1 norm for discrete features and L2 norm for continuous features (scaled by number of bits per feature).
2. Calculate __fingerprint distance__: Calculate the weighted sum of all feature distances (sum of feature weights equals 1).

![The `kissim` fingerprint comparison.](../_static/kissim_comparison.png)

_Figure 1_: The pairwise `kissim` fingerprint comparison.

Respective objects performing these calculates are the `FeatureDistances` and `FingerprintDistance` objects.
Furthermore, such distances can not only be generated between two fingerprints as described above but also in bulk for a set of fingerprints in an all-against-all comparison using the objects `FingerprintDistanceGenerator` and `FeatureDistancesGenerator`.

Let's take a look at the API logic in this table again:

| Action                                                         | Module       | Single calculation    | Bulk calculation               |
|----------------------------------------------------------------|--------------|-----------------------|--------------------------------|
| Encode structures as fingerprint                               | `encoding`   | `Fingerprint`         | `FingerprintGenerator`         |
| Compare fingerprint features (calculate feature distance**s**) | `comparison` | `FeatureDistances`    | `FeatureDistancesGenerator`    |
| Compare fingerprints (calculate fingerprint distance)          | `comparison` | `FingerprintDistance` | `FingerprintDistanceGenerator` |

In [1]:
from pathlib import Path

In [2]:
# Path to this notebook
HERE = Path(_dh[-1])  # noqa: F821

Set up local KLIFS session using the `opencadd.databases.klifs` module.

In [3]:
from opencadd.databases.klifs import setup_local

KLIFS_LOCAL = setup_local(HERE / "../../kissim/tests/data/KLIFS_download/")

## Select structure KLIFS IDs

In [4]:
structure_klifs_ids = [109, 118, 12347, 1641, 3833, 9122]

## Generate fingerprints

Let's generate a few fingerprints for the structures in our local KLIFS download using the bulk fingerprint generator `FingerprintGenerator`.

In [5]:
from kissim.encoding import FingerprintGenerator

fingerprint_generator = FingerprintGenerator.from_structure_klifs_ids(
    structure_klifs_ids=structure_klifs_ids, klifs_session=KLIFS_LOCAL
)
print(f"Number of fingerprints: {len(fingerprint_generator.data.keys())}")

Number of fingerprints: 6


__Note__: If fingerprint cannot be generated (e.g. because structural data is missing), the structure is skipped.

## Compare two fingerprints

Let's first focus on the comparison between two fingerprints only.

For two fingerprints (`Fingerprint` objects), we will 

1. Calculate the _feature distances_ using `FeatureDistances` and 
2. Calculate based on these _feature distances_ and given _feature weights_ the final _fingerprint distance_ using `FingerprintDistance`.

### Generate feature distances between two fingerprints (`FeatureDistances`)

- Input: Two `Fingerprint` objects
- Output: `FeatureDistances` object

In [6]:
fingerprints = list(fingerprint_generator.data.values())
fingerprint1 = fingerprints[0]
fingerprint2 = fingerprints[1]

In [7]:
from kissim.comparison import FeatureDistances

feature_distances = FeatureDistances.from_fingerprints(fingerprint1, fingerprint2)
print(f"Kinase pair: {feature_distances.kinase_pair_ids}")
print(f"Structure pair: {feature_distances.structure_pair_ids}")
feature_distances.data

Kinase pair: ('ABL2', 'ABL2')
Structure pair: (109, 118)


,feature_type,feature_name,distance,bit_coverage
0,physicochemical,size,0.000000,1.00
1,physicochemical,hbd,0.000000,1.00
2,physicochemical,hba,0.000000,1.00
3,physicochemical,charge,0.000000,1.00
4,physicochemical,aromatic,0.000000,1.00
5,physicochemical,aliphatic,0.000000,1.00
6,physicochemical,sco,0.080000,0.88
7,physicochemical,exposure,0.294118,1.00
8,distances,distance_to_centroid,0.059839,1.00
9,distances,distance_to_hinge_region,0.122168,1.00


### Generate fingerprint distance between two fingerprints (`FingerprintDistance`)

- Input: `FeatureDistances` object and optionally feature weights
- Output: `FingerprintDistance` object

#### Use standard feature weights

In [8]:
from kissim.comparison import FingerprintDistance

fingerprint_distance = FingerprintDistance.from_feature_distances(
    feature_distances, feature_weights=None
)
print(f"Fingerprint distance: {fingerprint_distance.distance}")
print(f"Fingerprint bit coverage: {fingerprint_distance.bit_coverage}")
print(f"Feature weights: {fingerprint_distance.feature_weights.round(3).tolist()}")

Fingerprint distance: 0.07421423894307076
Fingerprint bit coverage: 0.9919999999999999
Feature weights: [0.067, 0.067, 0.067, 0.067, 0.067, 0.067, 0.067, 0.067, 0.067, 0.067, 0.067, 0.067, 0.067, 0.067, 0.067]


#### Use user-defined feature weights

In [9]:
feature_weights = [0.3 / 8] * 8 + [0.5 / 4] * 4 + [0.2 / 3] * 3
fingerprint_distance = FingerprintDistance.from_feature_distances(
    feature_distances, feature_weights=feature_weights
)
print(f"Fingerprint distance: {fingerprint_distance.distance}")
print(f"Fingerprint bit coverage: {fingerprint_distance.bit_coverage}")
print(f"Feature weights: {fingerprint_distance.feature_weights.round(3).tolist()}")

Fingerprint distance: 0.08417398268335104
Fingerprint bit coverage: 0.9954999999999999
Feature weights: [0.038, 0.038, 0.038, 0.038, 0.038, 0.038, 0.038, 0.038, 0.125, 0.125, 0.125, 0.125, 0.067, 0.067, 0.067]


## Compare all-against-all fingerprints

Let's now take a look at the bulk distance generators to generate all-against-all comparisons for a set of fingerprints.

For a `FingerprintGenerator` object, which contains the fingerprints for a set of structures, we will 

1. Calculate _feature distances_ for all fingerprint pairs using `FeatureDistancesGenerator` and 
2. Calculate based on these _feature distances_ and given _feature weights_ the final _fingerprint distance_ for all fingerprint pairs using `FingerprintDistanceGenerator`.

### Generate feature distances for all pairwise structures/fingerprints (`FeatureDistancesGenerator`)

- Input: `FingerprintGenerator` object
- Output: `FeatureDistancesGenerator` object

In [10]:
from kissim.comparison import FeatureDistancesGenerator

feature_distances_generator = FeatureDistancesGenerator.from_fingerprint_generator(
    fingerprint_generator
)
print(f"One example structure pair: {feature_distances_generator.data[0].structure_pair_ids}")
feature_distances_generator.data[0]

One example structure pair: (109, 118)


### Generate fingerprint distance for all pairwise structures/fingerprints (`FingerprintDistanceGenerator`)

- Input: `FeatureDistancesGenerator` object and optionally feature weights
- Output: `FingerprintDistanceGenerator` object

In [11]:
from kissim.comparison import FingerprintDistanceGenerator

fingerprint_distance_generator = FingerprintDistanceGenerator.from_feature_distances_generator(
    feature_distances_generator
)

In [12]:
fingerprint_distance_generator.data.head(20)

,structure1,structure2,kinase1,kinase2,distance,coverage
0,109,118,ABL2,ABL2,0.074214,0.992000
1,109,12347,ABL2,BRAF,0.259053,0.919333
2,109,1641,ABL2,CHK1,0.253045,0.990667
3,109,3833,ABL2,AAK1,0.277368,0.990667
4,109,9122,ABL2,ADCK3,0.358882,0.990667
5,118,12347,ABL2,BRAF,0.273133,0.918000
6,118,1641,ABL2,CHK1,0.246844,0.989333
7,118,3833,ABL2,AAK1,0.282949,0.990000
8,118,9122,ABL2,ADCK3,0.360833,0.989333
9,12347,1641,BRAF,CHK1,0.303330,0.918000


#### Kinase distance matrix

In [13]:
fingerprint_distance_generator.kinase_distance_matrix(by="minimum")

kinase2,AAK1,ABL2,ADCK3,BRAF,CHK1
kinase1,,,,,
AAK1,0.000000,0.277368,0.303542,0.307277,0.229590
ABL2,0.277368,0.000000,0.358882,0.259053,0.246844
ADCK3,0.303542,0.358882,0.000000,0.376875,0.347142
BRAF,0.307277,0.259053,0.376875,0.000000,0.303330
CHK1,0.229590,0.246844,0.347142,0.303330,0.000000


Show on diagonal experimental values for structure pairs representing each kinase pair (as opposed to simply setting the diagonal to 0 by default).

In [14]:
fingerprint_distance_generator.kinase_distance_matrix(by="minimum", fill_diagonal=False)

kinase2,AAK1,ABL2,ADCK3,BRAF,CHK1
kinase1,,,,,
AAK1,NaN,0.277368,0.303542,0.307277,0.229590
ABL2,0.277368,0.074214,0.358882,0.259053,0.246844
ADCK3,0.303542,0.358882,NaN,0.376875,0.347142
BRAF,0.307277,0.259053,0.376875,NaN,0.303330
CHK1,0.229590,0.246844,0.347142,0.303330,NaN


More structure-kinase mapping methods are available, e.g. `maximum` or `mean`. Additionally, the number of structure pairs per kinase pair can be fetched.

In [15]:
fingerprint_distance_generator.kinase_distance_matrix(by="size")

kinase2,AAK1,ABL2,ADCK3,BRAF,CHK1
kinase1,,,,,
AAK1,0,2,1,1,1
ABL2,2,1,2,2,2
ADCK3,1,2,0,1,1
BRAF,1,2,1,0,1
CHK1,1,2,1,1,0


#### Structure distance matrix

In [16]:
fingerprint_distance_generator.structure_distance_matrix()

structure2,109,118,1641,3833,9122,12347
structure1,,,,,,
109,0.000000,0.074214,0.253045,0.277368,0.358882,0.259053
118,0.074214,0.000000,0.246844,0.282949,0.360833,0.273133
1641,0.253045,0.246844,0.000000,0.229590,0.347142,0.303330
3833,0.277368,0.282949,0.229590,0.000000,0.303542,0.307277
9122,0.358882,0.360833,0.347142,0.303542,0.000000,0.376875
12347,0.259053,0.273133,0.303330,0.307277,0.376875,0.000000
